In [1]:
import numpy as np
import tensorflow as tf
import os
import os.path as osp
import re

output_graph_name = 'constant_graph_weights_full_0.8.pb'
input_fld = 'input_fld_path'

def clean(s):
    s = re.sub(r'@\S+', r'', s)
    return re.sub(r'[^\x00-\x7f]', r'', s)

def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name="prefix")
    return graph


In [3]:
graph = load_graph('input_fld_pathtensorflow_model/' + output_graph_name)
# for op in graph.get_operations():
#     print(op.name)
char_indices = {'\t': 0, '\n': 1, '\r': 2, '!': 3, ' ': 4, '#': 5, '"': 6, '%': 7, '$': 8, "'": 9, '&': 10, ')': 11, '(': 12, '+': 13, '*': 14, '-': 15, ',': 16, '/': 17, '.': 18, '1': 19, '0': 20, '3': 21, '2': 22, '5': 23, '4': 24, '7': 25, '6': 26, '9': 27, '8': 28, ';': 29, ':': 30, '=': 31, '<': 32, '?': 33, '>': 34, '@': 35, '[': 36, ']': 37, '\\': 38, '_': 39, '^': 40, 'a': 41, '`': 42, 'c': 43, 'b': 44, 'e': 45, 'd': 46, 'g': 47, 'f': 48, 'i': 49, 'h': 50, 'k': 51, 'j': 52, 'm': 53, 'l': 54, 'o': 55, 'n': 56, 'q': 57, 'p': 58, 's': 59, 'r': 60, 'u': 61, 't': 62, 'w': 63, 'v': 64, 'y': 65, 'x': 66, '{': 67, 'z': 68, '}': 69, '|': 70, '\x7f': 71, '~': 72}
x = graph.get_tensor_by_name('prefix/input_1:0')
y = graph.get_tensor_by_name('prefix/output_node0:0')
kl = graph.get_tensor_by_name('prefix/lstm_1/keras_learning_phase:0')

maxlen = 512
max_sentences = 1

def processSentence(sentence):
    x = np.zeros(maxlen, dtype=np.int64)
    for t, char in enumerate(sentence[-maxlen:]):
        x[maxlen - 1 - t] = char_indices[char]
    return x
        
with tf.Session(graph=graph) as sess:
    sentences = [
        "The foot is really bad. Like the worst it's ever been. I can barely walk right now.",
        " It took me waaay too long to get your message about being ashamed...right now I really am ashamed",
        "thx for nothing on getting us out of the country and back to US. Broken plane? Come on. Get another one.",
        "too bad",
        "that sucks",
        "i'm disappointed by this performance",
        "this is a afewwfaw", 
        "huh?!@#!@",
        "wtf",
        "oh i'm so sorry",
        "this is good",
        "have a great trip",
        "This is such a great deal! Already thinking about my 2nd trip to @Australia "]
    input_x = np.ones((len(sentences), max_sentences, maxlen), dtype=np.int64) * -1
    for i, s in enumerate(sentences):
        input_x[i][0] = processSentence(clean(s.lower()))
    y_out = sess.run(y, feed_dict={
        x: input_x, kl: False
    })

    print(y_out)

[[ 0.05642456]
 [ 0.05464973]
 [ 0.05642456]
 [ 0.05642456]
 [ 0.05642456]
 [ 0.05642456]
 [ 0.05643943]
 [ 0.34382129]
 [ 0.31523278]
 [ 0.05642456]
 [ 0.95141006]
 [ 0.88607085]
 [ 0.84239691]]
